In [1]:
import surprise

import tensorflow as tf

from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split
import zipfile
from surprise import Reader, Dataset, SVD
from surprise import accuracy
import random
from random import randint

<h4> Read in data

In [49]:
# # Read data into an array of strings
with open('./ml-100k/u.data') as f:
    all_lines = f.readlines()

# Prepare the data to be used in Surprise
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file('./new_data.data', reader=reader)


# reader = Reader(line_format='user item rating timestamp', sep='\t')
# data = Dataset.load_from_file('./ml-100k/u.data', reader=reader)

In [3]:
def strip_content(data):
    return r_unwanted.sub(", ", data)

In [4]:
def create_dataframe(data):
    df_data = pd.DataFrame(data)

    df_data[0] = df_data[0].apply(strip_content)

    foo = lambda x: pd.Series([i for i in (x.split(','))])

    df_final = df_data[0].apply(foo)

    df_final.rename(columns={0:'userID', 1:'movieID', 2:'rating', 3: 'timestamp'}, 
                     inplace=True)

    df_final = df_final.drop(columns=[4])
    
    return df_final

In [5]:
# df_final = create_dataframe(all_lines)

In [6]:
# sim_options = {
#     "name": ["msd", "cosine"],
#     "min_support": [3, 4, 5],
#     "user_based": [False, True],
# }

# param_grid = {"sim_options": sim_options}

# gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> Grid search for best params </h4>

In [7]:
# data

In [29]:
# param_grid = {
#     "n_epochs": [5, 10],
#     "lr_all": [0.002, 0.005],
#     "reg_all": [0.4, 0.6]
# }
# gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> SVD algo chosen, with best params obtained from GS

In [16]:
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

<h4>Cross validation, 5 folds

In [17]:
# cross_validate(algo, data_other, measures=['RMSE', 'MAE'], cv=5, verbose=True)

<h4>Create train and test set, apply predictions on test set

In [26]:
def RMSE_predict_train_test(data):
    # sample random trainset and testset
    # test set is made of 25% of the ratings.
    trainset, testset = train_test_split(data, test_size=.25)

    # We'll use the famous SVD algorithm.
    algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

    # Train the algorithm on the trainset, and predict ratings for the testset
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Then compute RMSE
    return accuracy.rmse(predictions)

In [50]:
print("CREATING ALGO\n")
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

print("CALCULATING RMSE USING A TEST SET (1/4) OF TRAINING SET")
start = timeit.default_timer()
rmse = RMSE_predict_train_test(data)
stop = timeit.default_timer()
print('Time to calculate RMSE: ', stop - start, "\n")  

print("BUILDING TRAINING SET\n")
trainset = data.build_full_trainset()

print("TRAINING ALGORITHM")
start = timeit.default_timer()
algo.fit(trainset)
stop = timeit.default_timer()
print('Time to train alogrithm ', stop - start, "\n") 


print("PREDICTING RATINGS")
start = timeit.default_timer()

ratings = predict_scores(data, algo)

stop = timeit.default_timer()

print('Time to train alogrithm ', stop - start, "\n") 

 

CREATING ALGO

CALCULATING RMSE USING A TEST SET (1/4) OF TRAINING SET
RMSE: 1.4184
Time to calculate RMSE:  974.1952820000001 

BUILDING TRAINING SET

TRAINING ALGORITHM
Time to train alogrithm  1022.7955871000001 

PREDICTING RATINGS
user: 196        item: 302        r_ui = 4.00   est = 3.79   {'was_impossible': False}
Time to train alogrithm  4886.2979094 



In [51]:
len(ratings)

158685456

<h4> Training Time
   

In [19]:
import timeit

In [23]:
def training_time(data):
    start = timeit.default_timer()

    trainset = data.build_full_trainset()

    algo.fit(trainset)


    stop = timeit.default_timer()

    print('Time: ', stop - start)  

In [24]:
training_time(data)

Time:  5.9843355


<h4>Predict scores for all users for all movies. Test the speed of the predictions.

In [39]:
def predict_scores(data, algo):
        # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    # Create empty list to store predictions
    ratings = []
    
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings.append(prediction)
            
    print(algo.predict(str(196), str(302), 4))
    return ratings

<h4> Time to predict all ratings

In [38]:
import timeit

start = timeit.default_timer()


ratings = predict_scores(data, algo)
    
stop = timeit.default_timer()

print('Time: ', stop - start)  

user: 196        item: 302        r_ui = 4.00   est = 4.02   {'was_impossible': False}
Time:  21.4044990000001


<h4> Create Dataframe

In [ ]:
def predict_scores_df(data, df):
    # Build Training set. Needed to fit to create model.
    trainset = data.build_full_trainset()
    
    algo.fit(trainset)
    
    # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    # Create empty list to store predictions
    ratings = {}
    ratings_list = []
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings['userID'] = str(user_id)
            ratings['movieID'] = str(item_id)
            ratings['rating'] = prediction
            
            ratings_list.append(ratings)
            
            ratings = {}
            
    return ratings_list

In [ ]:
import timeit

start = timeit.default_timer()

predicted_scores_dict =  predict_scores_df(data, df_final)
    
stop = timeit.default_timer()

print('Time: ', stop - start)  


<h4> Dataframe with labels

In [ ]:
prediced_scores_dict = predict_scores_df(data, df_final)

In [ ]:
df_ratings_all_users = df_final.append(predicted_scores_dict)

In [ ]:
import random

In [ ]:
# def create_data(data):
#     trainset = data.build_full_trainset()
    
#     user_ids = trainset.all_users()
    
#     data_list = []
#     for user_id in user_ids:
#         for movie in range(item_ids[-1]+1, item_ids[-1]*100):

#             new_data = {}

#             new_data['userID'] = str(user_id)
#             new_data['movieID'] = str(movie)
#             new_data['rating'] = random.randint(1,5)

#             data_list.append(new_data)
#     return data_list

In [ ]:
# def create_data_other(data):
#     trainset = data.build_full_trainset()

#     user_ids = trainset.all_users()
#     item_ids = trainset.all_items()

#     data_list = []
#     for user_id in user_ids:
#         for movie in range(item_ids[-1]+1, item_ids[-1]*60):

#             new_data = str(user_id)+'\t'+str(movie)+'\t'+str(random.randint(1,5))+'\t'+'NaN\n'

#             data_list.append(new_data)
#     return data_list

<h4> Create extended dataset

In [ ]:
def create_data_other(data):
    trainset = data.build_full_trainset()

    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    data_list = []
    # Create new movies (168200 in total)
    for movie in range(item_ids[-1]+1, item_ids[-1]*100):
        # For every movie, there will be 100 users rating the movie
        user_generated = [randint(0, user_ids[-1]) for p in range(0, 100)]
        for user in user_generated:
            # Create a random generated score for the movies
            new_data = str(user)+'\t'+str(movie)+'\t'+str(random.randint(1,5))+'\t'+'NaN\n'

            data_list.append(new_data)
    return data_list

In [ ]:
data_newest = create_data_other(data_other)

Create the new data (as a list)

In [ ]:
new_data = []

new_data.extend(all_lines)
new_data.extend(data_newest)

Check difference in sizes


In [ ]:
len(data_newest)/len(all_lines)

Save file on filesystem

In [ ]:
# # Read data into an array of strings
with open('./new_data.data') as f:
    all_lines = f.readlines()

Load data

In [ ]:

# Prepare the data to be used in Surprise
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file('./new_data.data', reader=reader)

Calculate prediction time

In [ ]:
import timeit

start = timeit.default_timer()


ratings = predict_scores(data)
    
stop = timeit.default_timer()

print('Time: ', stop - start)  